# Single Particle performance plots


   * [GEN Level plots](#Gen-level)
   * [All clusters without selection](#All-clusters-without-selection)
      * [2D clusters per layer](#2D-clusters-per-layer)
      * [2D cluster energy](#2D-cluster-energy)
      * [2D cluster: # of TriggerCells](#2D-cluster:-#-of-TriggerCells)
      * [# of 3D clusters](##-of-3D-clusters)
   * [Cluster matched to GEN particles](#Cluster-matched-to-GEN-particles)
      * [2D clusters per layer](#2D-clusters-per-layer)
      * [2D clusters: # of Trigger Cells](#2D-clusters:-#-of-Trigger-Cells)
      * [2D Cluster Energy](#2D-Cluster-Energy)
      * [3D clusters: # of 2D components](#3D-clusters:-#-of-2D-components)
      * [3D cluster energy](#3D-cluster-energy)
   * [Resolution studies](#Resolution-Studies)
      * [3D cluster: pt resolution](#3D-cluster:-Pt-resolution)
      * [2D position resolution](#Postion-resolution-of-2D-clusters)
   * [Trigger Tower performance](#Trigger-Tower-plots)
      * [Towers matched to GEN](#Towers-matched-to-GEN-particles)
      * [Resolution studies](#Trigger-Tower-resolution)



In [1]:
import sys
sys.path.insert(0, '..')
import ROOT
import root_numpy as rnp
import python.l1THistos as histos
import math
import uuid
import pandas as pd

ROOT.enableJSVis()
#ROOT.enableJSVis()

#from drawingTools import *

normalized_histos = list()

Welcome to JupyROOT 6.14/02


In [3]:
# %load ../drawingTools
import ROOT
import math
import uuid

# some useful globals, mainly to deal with ROOT idiosyncrasies
c_idx = 0
p_idx = 0
colors = range(1, 12)
stuff = []
f_idx = 0

ROOT.gStyle.SetOptTitle(False)
ROOT.gStyle.SetPadBottomMargin(0.13)
ROOT.gStyle.SetPadLeftMargin(0.13)
ROOT.gStyle.SetPadRightMargin(0.13)

# ROOT.gStyle.SetCanvasBorderMode(0)
# ROOT.gStyle.SetCanvasColor(0)


def getText(text, ndc_x, ndc_y):
    global stuff
    rtext = ROOT.TLatex(ndc_x, ndc_y, text)
    stuff.append(rtext)
    rtext.SetNDC(True)
    # rtext.SetTextFont(40)
    rtext.SetTextSize(0.03)
    return rtext

def getLegend(x1=0.7, y1=0.71, x2=0.95, y2=0.85):
    global stuff
    legend = ROOT.TLegend(x1, y1, x2, y2)
    stuff.append(legend)
    legend.SetFillColor(0)
    legend.SetFillStyle(0)
    legend.SetBorderSize(0)
    legend.SetTextSize(0.05)
    return legend

def newCanvas(name=None, title=None, height=600, width=800, xdiv=0, ydiv=0, form=4):
    global c_idx
    if name is None:
        name = 'c_{}'.format(uuid.uuid4().hex[:6])
        c_idx += 1
    if title is None:
        title = name
    # print name, title, width, height
    canvas = ROOT.TCanvas(name, title, width, height)
    if(xdiv*ydiv != 0):
        canvas.Divide(xdiv, ydiv)
    global stuff
    stuff.append(canvas)
    return canvas

def drawAndProfileX(plot2d, miny=None, maxy=None, do_profile=True, options='', text=None):
    global p_idx
    if miny and maxy:
        plot2d.GetYaxis().SetRangeUser(miny, maxy)
    c = newCanvas()
    c.SetGrid(1, 1)
    c.cd()
    plot2d.Draw(options)
    ROOT.gPad.SetGrid(1, 1)
    ROOT.gStyle.SetGridColor(15)

    if do_profile:
        profname = plot2d.GetName()+'_prof_'+str(p_idx)
        p_idx += 1
        firstbin = 1
        lastbin = -1
        prof = plot2d.ProfileX(profname, firstbin, lastbin, 's')
        prof.SetMarkerColor(2)
        prof.SetLineColor(2)
        prof.Draw('same')

    if text:
        rtext = getText(text, 0.15, 0.85)
        rtext.Draw('same')

    c.Draw()


def draw(histograms,
         labels,
         options='',
         norm=False,
         logy=False,
         min_y=None,
         max_y=None,
         text=None,
         y_axis_label=None,
         x_axis_label=None,
         v_lines=None,
         h_lines=None,
         do_stats=False,
         do_profile=False,
         do_ratio=False):
    global colors
    global stuff
    global p_idx


    # 0 - determine kind and # of histos
    n_histos = len(histograms)
    if n_histos == 0:
        print '[draw]**ERROR: empy histo list'
        return

    histo_class = histograms[0].ClassName()

    # 1 - check argument consisntency
    if 'TH2' not in histo_class and do_profile:
        print '[draw]**ERROR: do_profile set for histo of class: {}'.format(histo_class)
        return

    # TH1 histograms are overlayed
    # TH2 histograms are drawn side by side in the same canvas
    overlay = True
    if n_histos > 1:
        if 'TH2' in histo_class or 'TH3' in histo_class:
            overlay = False

    # 2 - determine canvas properties
    xdiv = 0
    ydiv = 0
    c_width = 800
    c_height = 600
    # do_ratio = False

    if do_ratio:
        c_height = 800

    if not overlay:
        xdiv = 2
        c_width = 1000.
        ydiv = math.ceil(float(n_histos)/2)
        c_height = 500*ydiv
    canvas = newCanvas(name=None,
                       title=None,
                       height=int(c_height),
                       width=int(c_width),
                       xdiv=int(xdiv),
                       ydiv=int(ydiv))
    if overlay:
        canvas.SetRightMargin(0.30)

    canvas.cd()
    leg = getLegend()

    drawn_histos = []
    # drawn_histos = histograms
    for hidx, hist in enumerate(histograms):
        opt = options
        if 'TGraph' not in histo_class:
            hist.SetStats(do_stats)
        else:
            opt = 'P'+options
        if overlay:
            hist.SetLineColor(colors[hidx])
            if 'TGraph' in histo_class:
                hist.SetMarkerColor(colors[hidx])
            if hidx:
                opt = 'same,'+opt
            else:
                if 'TGraph' in histo_class:
                    opt = opt+'A'
        else:
            canvas.cd(hidx+1)
        # print hidx, opt
        d_hist = hist
        if norm:
            d_hist = hist.DrawNormalized(opt, 1.)
        else:
            hist.Draw(opt)

        if do_profile:
            profname = d_hist.GetName()+'_prof_'+str(p_idx)
            p_idx += 1
            firstbin = 1
            lastbin = -1
            prof = d_hist.ProfileX(profname, firstbin, lastbin, 's')
            prof.SetMarkerColor(2)
            prof.SetLineColor(2)
            prof.Draw('same')

        if overlay:
            if 'TGraph' not in histo_class:
                leg.AddEntry(hist, labels[hidx], 'l')
            else:
                leg.AddEntry(hist, labels[hidx], 'P')
        else:
            if text:
                newtext = '{}: {}'.format(labels[hidx], text)
                rtext = getText(newtext, 0.15, 0.85)
                rtext.Draw('same')
        drawn_histos.append(d_hist)

    if do_ratio:
        h_ratio = ROOT.TRatioPlot(drawn_histos[0], drawn_histos[1])
        stuff.append(h_ratio)
        h_ratio.Draw()

        pad = canvas.cd(1)
        h_ratio.GetUpperPad().SetBottomMargin(0)
        h_ratio.GetUpperPad().SetRightMargin(0.3)
        if logy:
            h_ratio.GetUpperPad().SetLogy()
        pad.Update()
        pad = canvas.cd(2)
        h_ratio.GetLowerPad().SetTopMargin(0)
        h_ratio.GetLowerPad().SetRightMargin(0.3)
        pad.Update()
        # hratio.GetLowerRefGraph().GetYaxis().SetRangeUser(-6, 6)

    canvas.Update()

    # we now set the axis properties
    max_value = max_y
    min_value = min_y

    if min_y is None:
        min_value = min([hist.GetBinContent(hist.GetMinimumBin()) for hist in drawn_histos])
    if max_y is None:
        max_value = max([hist.GetBinContent(hist.GetMaximumBin()) for hist in drawn_histos])*1.2
    # print min_value, max_value

    for hist in drawn_histos:
        hist.GetYaxis().SetRangeUser(min_value, max_value)
        if y_axis_label:
            hist.GetYaxis().SetTitle(y_axis_label)
        if x_axis_label:
            hist.GetXaxis().SetTitle(x_axis_label)

    canvas.Draw()

    # we draw additional stuff if needed
    if overlay:
        canvas.cd()
        leg.Draw()
        if text:
            rtext = getText(text, 0.15, 0.85)
            rtext.Draw("same")

    pad_range = range(0, 1)
    if not overlay:
        pad_range = range(1, len(histograms)+1)

    for pad_id in pad_range:
        canvas.cd(pad_id)
        if v_lines:
            for v_line_x in v_lines:
                aline = ROOT.TLine(v_line_x, ROOT.gPad.GetUymin(), v_line_x,  ROOT.gPad.GetUymax())
                aline.SetLineStyle(2)
                aline.Draw("same")
                stuff.append(aline)
        if h_lines:
            for h_line_y in h_lines:
                aline = ROOT.TLine(ROOT.gPad.GetUxmin(), h_line_y, ROOT.gPad.GetUxmax(),  h_line_y)
                aline.SetLineStyle(2)
                aline.Draw("same")
                stuff.append(aline)
        if logy:
            if not do_ratio:
                ROOT.gPad.SetLogy()

        ROOT.gPad.Update()
    canvas.Update()
    canvas.Draw()

    return canvas


In [11]:
# %load samples.py
import ROOT
import pandas as pd
import python.selections as selections

version = 'v66'

files = {}
file_keys = {}


class RootFile:
    def __init__(self, file_name):
        global file
        self.file_name = file_name
        if self.file_name not in files.keys():
            print 'get file: {}'.format(self.file_name)
            files[self.file_name] = ROOT.TFile(self.file_name)
        self._file = files[self.file_name]
        self._file_keys = None

    def cd(self):
        self._file.cd()

    def GetListOfKeys(self):
        global file_keys
        if self.file_name not in file_keys.keys():
            print 'get list'
            file_keys[self.file_name] = self._file.GetListOfKeys()
        self._file_keys = file_keys[self.file_name]
        return self._file_keys


class Sample():
    def __init__(self, name, label, version=None, type=None):
        self.name = name
        self.label = label
        if version:
            version = '_'+version
        else:
            version = ''
        self.histo_filename = '../plots1/histos_{}{}.root'.format(self.name, version)
        self.histo_file = ROOT.TFile(self.histo_filename, 'r')
        self.type = type


# sample_names = ['ele_flat2to100_PU0',
#                 'ele_flat2to100_PU200',
#                 'photonPt35_PU0',
#                 'photonPt35_PU200']


def get_label_dict(selections):
    dictionary = {}
    for sel in selections:
        dictionary[sel.name] = sel.label
    return dictionary


class HProxy:
    def __init__(self, classtype, tp, tp_sel, gen_sel, root_file):
        self.classtype = classtype
        self.tp = tp
        self.tp_sel = tp_sel
        self.gen_sel = gen_sel
        self.root_file = root_file
        self.instance = None

    def get(self):
        if self.instance is None:
            name = '{}_{}_{}'.format(self.tp, self.tp_sel, self.gen_sel)
            if self.gen_sel is None:
                name = '{}_{}'.format(self.tp, self.tp_sel)
            self.instance = self.classtype(name, self.root_file)
        return self.instance


class HPlot:
    def __init__(self, samples, tp_sets, tp_selections, gen_selections):
        self.tp_sets = tp_sets
        self.tp_selections = tp_selections
        self.gen_selections = gen_selections
        self.pus = []
        self.labels_dict = {}

        for sample in samples:
            self.pus.append(sample.label)
            self.labels_dict[sample.type] = sample.type

        self.data = pd.DataFrame(columns=['sample', 'pu', 'tp', 'tp_sel', 'gen_sel', 'classtype', 'histo'])

        self.labels_dict.update(tp_sets)
        self.labels_dict.update(tp_selections)
        self.labels_dict.update(gen_selections)
        self.labels_dict.update({'PU0': 'PU0', 'PU200': 'PU200'})

    def cache_histo(self,
                    classtype,
                    samples,
                    pus,
                    tps,
                    tp_sels,
                    gen_sels):
        if gen_sels is None:
            gen_sels = [None]

        for sample in samples:
            print sample
            for tp in tps:
                for tp_sel in tp_sels:
                    for gen_sel in gen_sels:
                        print sample, tp, tp_sel, gen_sel
                        self.data = self.data.append({'sample': sample.type,
                                                      'pu': sample.label,
                                                      'tp': tp,
                                                      'tp_sel': tp_sel,
                                                      'gen_sel': gen_sel,
                                                      'classtype': classtype,
                                                      'histo': HProxy(classtype, tp, tp_sel, gen_sel, sample.histo_file)},
                                                     ignore_index=True)

    def get_histo(self,
                  classtype,
                  sample=None,
                  pu=None,
                  tp=None,
                  tp_sel=None,
                  gen_sel=None,
                  debug=False):
        histo = None
        labels = []
        text = ''

        query = '(pu == @pu) & (tp == @tp) & (tp_sel == @tp_sel) & (classtype == @classtype)'
        if gen_sel is not None:
            query += ' & (gen_sel == @gen_sel)'
        else:
            query += ' & (gen_sel.isnull())'
        if sample is not None:
            query += '& (sample == @sample)'

        histo_df = self.data.query(query)

        if histo_df.empty:
            print 'No match found for: pu: {}, tp: {}, tp_sel: {}, gen_sel: {}, classtype: {}'.format(pu, tp, tp_sel, gen_sel, classtype)
            return None, None, None
        if debug:
            print histo_df

        field_counts = histo_df.apply(lambda x: len(x.unique()))
        label_fields = []
        text_fields = []
        # print field_counts
        for field in field_counts.iteritems():
            if(field[1] > 1 and field[0] != 'histo'):
                label_fields.append(field[0])
            if(field[1] == 1 and field[0] != 'histo' and field[0] != 'classtype' and field[0] != 'sample'):
                if(gen_sel is None and field[0] == 'gen_sel'):
                    continue
                text_fields.append(field[0])
#         print 'label fields: {}'.format(label_fields)
#         print 'text fields: {}'.format(text_fields)

        for item in histo_df[label_fields].iterrows():
            labels.append(', '.join([self.labels_dict[tx] for tx in item[1].values if self.labels_dict[tx] != '']))

        # print labels
        text = ', '.join([self.labels_dict[fl] for fl in histo_df[text_fields].iloc[0].values if self.labels_dict[fl] != ''])
        histo = [his.get() for his in histo_df['histo'].values]
        return histo, labels, text


# -------------------------------------------------------------------------

samples_ele = [
    Sample('ele_flat2to100_PU0', 'PU0', version, 'V8'),
    Sample('ele_flat2to100_PU200', 'PU200', version+'t', 'V8')
    ]


samples_ele_v9 = [
    Sample('all_flat5to80_PU0', 'PU0', 'v67t', 'V9'),
    Sample('all_flat5to80_PU200', 'PU200', 'v67t', 'V9')
    ]

samples_photons = [
    Sample('photon_flat8to150_PU0', 'PU0', version, 'photon'),
    Sample('photon_flat8to150_PU200', 'PU200', version, 'photon')
    ]

samples_pions = [
    Sample('pion_flat2to100_PU0', 'PU0', version, 'pions'),
    ]

samples_nugus = [
    Sample('nugun_alleta_pu0', 'PU0', version, 'mb'),
    Sample('nugun_alleta_pu200', 'PU200', version, 'mb')
    ]

samples_nugunrates = [
    Sample('nugun_alleta_pu200', 'PU200', version, 'mb')
    ]

tpsets = {'DEF': 'NNDR',
#           'DEFCalib': 'NNDR Calib v1'
         }

tpset_selections = {}
gen_selections = {}
samples = []

# tpset_selections.update(get_label_dict(tp_rate_selections))
tpset_selections.update(get_label_dict(selections.tp_match_selections))
gen_selections.update(get_label_dict(selections.gen_part_selections))
gen_selections.update({'nomatch': ''})


Error in <TFile::TFile>: file ../plots1/histos_pion_flat2to100_PU0_v66.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_nugun_alleta_pu0_v66.root does not exist


In [14]:
gen_selections = {'nomatch': ''}
tpset_selections = {'all': '',
                    'Em': 'EGId'}
samples = samples_ele
samples += samples_ele_v9
# samples += samples_pions

In [15]:
tpsets

{'DEF': 'NNDR'}

In [9]:
tpset_selections

{'Em': 'EGId', 'all': ''}

In [10]:
gen_selections

{'nomatch': ''}

In [16]:
%%time

hplot = HPlot(samples, tpsets, tpset_selections, gen_selections)


# hplot.cache_histo(classtype=histos.TriggerTowerHistos, 
#                   samples=samples,
#                   pus=[],
#                   tps=tpsets,
#                   tp_sels=tpset_selections,
#                   gen_sels=['nomatch'])


hplot.cache_histo(classtype=histos.HistoSetClusters, 
                  samples=samples,
                  pus=[],
                  tps=tpsets,
                  tp_sels=tpset_selections,
                  gen_sels=gen_selections)


<__main__.Sample instance at 0x11e6c6c20>
<__main__.Sample instance at 0x11e6c6c20> DEF Em nomatch
<__main__.Sample instance at 0x11e6c6c20> DEF all nomatch
<__main__.Sample instance at 0x11e6c6b48>
<__main__.Sample instance at 0x11e6c6b48> DEF Em nomatch
<__main__.Sample instance at 0x11e6c6b48> DEF all nomatch
<__main__.Sample instance at 0x11e6c6bd8>
<__main__.Sample instance at 0x11e6c6bd8> DEF Em nomatch
<__main__.Sample instance at 0x11e6c6bd8> DEF all nomatch
<__main__.Sample instance at 0x11e6c6cb0>
<__main__.Sample instance at 0x11e6c6cb0> DEF Em nomatch
<__main__.Sample instance at 0x11e6c6cb0> DEF all nomatch
CPU times: user 56.9 ms, sys: 9.37 ms, total: 66.3 ms
Wall time: 63.3 ms


In [17]:
hplot.data

,sample,pu,tp,tp_sel,gen_sel,classtype,histo
0,V8,PU0,DEF,Em,nomatch,python.l1THistos.HistoSetClusters,<__main__.HProxy instance at 0x11e64ccf8>
1,V8,PU0,DEF,all,nomatch,python.l1THistos.HistoSetClusters,<__main__.HProxy instance at 0x10d4b1c20>
2,V8,PU200,DEF,Em,nomatch,python.l1THistos.HistoSetClusters,<__main__.HProxy instance at 0x11e64c488>
3,V8,PU200,DEF,all,nomatch,python.l1THistos.HistoSetClusters,<__main__.HProxy instance at 0x11e68b9e0>
4,V9,PU0,DEF,Em,nomatch,python.l1THistos.HistoSetClusters,<__main__.HProxy instance at 0x11e69d830>
5,V9,PU0,DEF,all,nomatch,python.l1THistos.HistoSetClusters,<__main__.HProxy instance at 0x11e694d40>
6,V9,PU200,DEF,Em,nomatch,python.l1THistos.HistoSetClusters,<__main__.HProxy instance at 0x11e694fc8>
7,V9,PU200,DEF,all,nomatch,python.l1THistos.HistoSetClusters,<__main__.HProxy instance at 0x11e69dcb0>


## Electron flat pT gun 2 < pt < 100 GeV

In [22]:
sample_type = ['V8', 'V9']
pus =  ['PU0']
tps = ['DEF']
tp_select = ['all', 'Em']

### 3D clusters not matched to GEN

In [23]:
for tp in tps:
    for tp_sel in tp_select:
        hsets, labels, text = hplot.get_histo(histos.HistoSetClusters, sample_type, pus, tp, tp_sel, ['nomatch'])            
#         print hsets
        draw([his.hcl3d.h_pt for his in hsets], labels, norm=True, text=text, logy=True)


Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale


In [25]:
for tp in tps:
    for tp_sel in tp_select:
        hsets, labels, text = hplot.get_histo(histos.HistoSetClusters, sample_type, pus, tp, tp_sel, ['nomatch'])            
#         print hsets
        draw([his.hcl3d.h_eta for his in hsets], labels, norm=True, text=text, logy=True)


Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale


In [27]:
for tp in tps:
    for tp_sel in tp_select:
        hsets, labels, text = hplot.get_histo(histos.HistoSetClusters, sample_type, pus, tp, tp_sel, ['nomatch'])            
#         print hsets
        draw([his.hcl3d.h_bdt for his in hsets], labels, norm=True, text=text, logy=True)


AttributeError: Cluster3DHistos instance has no attribute 'h_bdt'

In [14]:

samples = samples_ele
tp_sets = ['DEF', 'DEF_em']
particles = ['nomatch']
labels = [sample.label for sample in samples]

histo_hGenDen_df = pd.DataFrame(index=tp_sets, columns=particles)
histo_hcl3d_df = pd.DataFrame(index=tp_sets, columns=particles)
histo_hcl2d_df = pd.DataFrame(index=tp_sets, columns=particles)
histo_htc_df = pd.DataFrame(index=tp_sets, columns=particles)
titles_df = pd.DataFrame(index=tp_sets, columns=particles)


for tp_set in tp_sets:
    for particle in particles:
        histo_hGenDen_df.loc[tp_set][particle] =  [histos.HistoSetEff('{}_{}'.format(tp_set, particle), sample.histo_file).h_den for sample in samples]
        histo_hcl3d_df.loc[tp_set][particle] =  [histos.HistoSetClusters('{}_{}'.format(tp_set, particle), sample.histo_file).hcl3d for sample in samples]
        histo_hcl2d_df.loc[tp_set][particle] =  [histos.HistoSetClusters('{}_{}'.format(tp_set, particle), sample.histo_file).hcl2d for sample in samples]
        histo_htc_df.loc[tp_set][particle] =  [histos.HistoSetClusters('{}_{}'.format(tp_set, particle), sample.histo_file).htc for sample in samples]
        titles_df.loc[tp_set, particle] = '{}, {}'.format(tpset_labels[tp_set], particle_labels[particle])


        
        
histo_hGenDen_df    
histo_hcl3d_df
histo_hcl2d_df
histo_htc_df


NameError: name 'tpset_labels' is not defined

In [ ]:
dir(histo_hcl3d_df.loc['DEF', 'ele'][0])
dir(histo_hcl2d_df.loc['DEF', 'ele'][0])
dir(histo_htc_df.loc['DEF', 'ele'][0])

### Gen level particle distribution

In [ ]:
tp_set = 'DEF'
particle = 'nomatch'

h_eta = [histo.h_eta for histo in histo_hGenDen_df.loc[tp_set, particle]]
drawSame(h_eta, labels, text=titles_df.loc[tp_set, particle])
h_pt = [histo.h_pt for histo in histo_hGenDen_df.loc[tp_set, particle]]
drawSame(h_pt, labels, text=titles_df.loc[tp_set, particle])
h_reachedEE = [histo.h_reachedEE for histo in histo_hGenDen_df.loc[tp_set, particle]]
drawSame(h_reachedEE, labels, text=titles_df.loc[tp_set, particle])


#### 3D clusters matched to GEN

In [ ]:

for particle in particles:
    for tp_set in tp_sets:
        hByTPset = [histo.h_pt for histo in histo_hcl3d_df.loc[tp_set, particle]]
        drawSame(hByTPset, labels, text=titles_df.loc[tp_set, particle], norm=True)


In [ ]:

for particle in ['nomatch']:
    for tp_set in tp_sets:
        hByTPset = [histo.h_eta for histo in histo_hcl3d_df.loc[tp_set, particle]]
        drawSame(hByTPset, labels, text=titles_df.loc[tp_set, particle])

3D cluster # of 2D components

In [ ]:

for particle in particles:
    for tp_set in ['DEF', 'DEF_em']:
        hByTPset = [histo.h_nclu for histo in histo_hcl3d_df.loc[tp_set, particle]]
        drawSame(hByTPset, labels, text=titles_df.loc[tp_set, particle])


In [ ]:

for particle in particles:
    for tp_set in ['DEF', 'DEF_em']:
        hByTPset = [histo.h_ncluVpt for histo in histo_hcl3d_df.loc[tp_set, particle]]
        drawSeveral(hByTPset, labels, options='COLZ', do_profile=True, text=titles_df.loc[tp_set, particle])


In [ ]:

for particle in particles:
    for tp_set in ['DEF']:
        hByTPset = [histo.h_iso0p2 for histo in histo_hcl3d_df.loc[tp_set, particle]]
        drawSame(hByTPset, labels, text=titles_df.loc[tp_set, particle], logy=True)


In [ ]:

for particle in particles:
    for tp_set in ['DEF']:
        hByTPset = [histo.h_isoRel0p2 for histo in histo_hcl3d_df.loc[tp_set, particle]]
        drawSame(hByTPset, labels, text=titles_df.loc[tp_set, particle], logy=True)

particle = 'ele'
tp_set = 'DEF'
hByTPset = [histo.h_isoRel0p2 for histo in histo_hcl3d_df.loc[tp_set, particle]]
draw(hByTPset[1])

#### 2D cluster components of matched 3D clusters

In [ ]:
histo_hcl2d_df

distribution of the counts of 2D cluster per layer 

In [ ]:
for particle in particles:
    for tp_set in tp_sets:
        hByTPset = [histo.h_layer for histo in histo_hcl2d_df.loc[tp_set, particle]]
        drawSame(hByTPset, labels, norm=True, text=titles_df.loc[tp_set, particle])
#         drawSeveral(hByTPset, labels, options='COLZ', do_profile=True)


Number of TCs composing the 2D cluster

In [ ]:
for particle in particles:
    for tp_set in tp_sets:
        hByTPset = [histo.h_ncells for histo in histo_hcl2d_df.loc[tp_set, particle]]
        drawSame(hByTPset, labels, norm=True, text=titles_df.loc[tp_set, particle])
#         drawSeveral(hByTPset, labels, options='COLZ', do_profile=True)


In [ ]:
for particle in particles:
    for tp_set in tp_sets:
        hByTPset = [histo.h_energy for histo in histo_hcl2d_df.loc[tp_set, particle]]
        drawSame(hByTPset, labels, norm=True, text=titles_df.loc[tp_set, particle])
#         drawSeveral(hByTPset, labels, options='COLZ', do_profile=True)

*NOTE*: we can conclude that at high PU the 3D clusters have more clusters in the first layers and the size (in terms of constituents and hence collected energy) of the 2D clusters grows

In the following we try to breakdown this behavior by layer


Distribution of the # of TC in the 2D cluster components vs layer

In [ ]:
for particle in particles:
    for tp_set in tp_sets:
        hByTPset = [histo.h_layerVncells for histo in histo_hcl2d_df.loc[tp_set, particle]]
#        drawSame(hByTPset, labels, norm=True)
        drawSeveral(hByTPset, labels, options='COLZ', do_profile=True, text=titles_df.loc[tp_set, particle])
        hByTPset = [histo.h_layerVncells.ProfileX(uuid.uuid4().hex[:6]) for histo in histo_hcl2d_df.loc[tp_set, particle]]
        drawSame(hByTPset, labels, text=titles_df.loc[tp_set, particle])

In [ ]:
for particle in particles:
    for tp_set in tp_sets:
        hByTPset = [histo.h_layerVenergy for histo in histo_hcl2d_df.loc[tp_set, particle]]
#        drawSame(hByTPset, labels, norm=True)
        drawSeveral(hByTPset, labels, options='COLZ', do_profile=True)
        hByTPset = [histo.h_layerVenergy.ProfileX(uuid.uuid4().hex[:6]) for histo in histo_hcl2d_df.loc[tp_set, particle]]
        drawSame(hByTPset, labels, text=titles_df.loc[tp_set, particle])

#### TCs components of Matched 3D clusters

In [ ]:
histo_htc_df

*WARNING*: looking inclusively at the energy of TCs is very misleading since PU TCs are mostly at high-eta one might conclude that they are more energetic....while they ahave the same pT

In [ ]:
for particle in particles:
    for tp_set in tp_sets:
        hByTPset = [histo.h_energy for histo in histo_htc_df.loc[tp_set, particle]]
        drawSame(hByTPset, labels, norm=True, text=titles_df.loc[tp_set, particle])


In [ ]:
for particle in particles:
    for tp_set in tp_sets:
        hByTPset = [histo.h_mipPt for histo in histo_htc_df.loc[tp_set, particle]]
        drawSame(hByTPset, labels, norm=True, text=titles_df.loc[tp_set, particle])

In [ ]:
for particle in particles:
    for tp_set in tp_sets:
        hByTPset = [histo.h_layer for histo in histo_htc_df.loc[tp_set, particle]]
        drawSame(hByTPset, labels, norm=True, text=titles_df.loc[tp_set, particle])